In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 264 kB in 6s (47.3 kB/s)
Reading

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ReadwriteVal").getOrCreate() 
spark

nyc_air_bnb.csv  spark-2.3.1-bin-hadoop2.7	spark-2.3.1-bin-hadoop2.7.tgz.1
sample_data	 spark-2.3.1-bin-hadoop2.7.tgz	spark-warehouse


In [3]:
from pyspark.sql.functions import to_date, col, max as max_, expr

In [4]:
airbnb = spark.read.csv('nyc_air_bnb.csv',inferSchema=True,header=True)
airbnb.show(5)

+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|  host_name|neighbourhood_group|neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+-----------+-------------------+-------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|       John|           Brooklyn|   Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca...|   2845|   Jennifer|          Manhatt

In [5]:
airbnb = airbnb[['minimum_nights','last_review', 'neighbourhood_group']]
airbnb = airbnb.selectExpr("minimum_nights as id", "last_review as date", "neighbourhood_group as class")
airbnb.show(5)

+---+----------+---------+
| id|      date|    class|
+---+----------+---------+
|  1|2018-10-19| Brooklyn|
|  1|2019-05-21|Manhattan|
|  3|      null|Manhattan|
|  1|2019-07-05| Brooklyn|
| 10|2018-11-19|Manhattan|
+---+----------+---------+
only showing top 5 rows



In [6]:
airbnb = airbnb.withColumn('date', to_date('date', 'yyyy-dd-mm'))
airbnb.show(5)

+---+----------+---------+
| id|      date|    class|
+---+----------+---------+
|  1|2018-01-10| Brooklyn|
|  1|2019-01-05|Manhattan|
|  3|      null|Manhattan|
|  1|2019-01-07| Brooklyn|
| 10|2018-01-11|Manhattan|
+---+----------+---------+
only showing top 5 rows



In [7]:
airbnb.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- class: string (nullable = true)



In [8]:
max_date = airbnb.withColumn("date", col("date").cast("timestamp")).groupBy("id").agg(max_("date"))
max_date.show(5)

+---+-------------------+
| id|          max(date)|
+---+-------------------+
|  7|2019-01-07 00:00:00|
| 51|               null|
| 15|2019-01-07 00:00:00|
|200|2019-01-07 00:00:00|
| 11|2019-01-07 00:00:00|
+---+-------------------+
only showing top 5 rows



In [9]:
left_join = airbnb.join(max_date, ["id"],"left")
left_join.show(5)

+---+----------+---------+-------------------+
| id|      date|    class|          max(date)|
+---+----------+---------+-------------------+
|  1|2018-01-10| Brooklyn|2019-01-07 00:00:00|
|  1|2019-01-05|Manhattan|2019-01-07 00:00:00|
|  3|      null|Manhattan|2019-01-07 00:00:00|
|  1|2019-01-07| Brooklyn|2019-01-07 00:00:00|
| 10|2018-01-11|Manhattan|2019-01-07 00:00:00|
+---+----------+---------+-------------------+
only showing top 5 rows



In [10]:
# adding cols last 4 weeks, last 6 weeks, last 8 weeks ....
import pyspark.sql.functions as F 

left_join = left_join.withColumn('last4weeks', F.date_sub(left_join['max(date)'], 30))
left_join = left_join.withColumn('last6weeks', F.date_sub(left_join['max(date)'], 45))
left_join = left_join.withColumn('last8weeks', F.date_sub(left_join['max(date)'], 60))

In [11]:
left_join.show(5)

+---+----------+---------+-------------------+----------+----------+----------+
| id|      date|    class|          max(date)|last4weeks|last6weeks|last8weeks|
+---+----------+---------+-------------------+----------+----------+----------+
|  1|2018-01-10| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  1|2019-01-05|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  3|      null|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  1|2019-01-07| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 10|2018-01-11|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
+---+----------+---------+-------------------+----------+----------+----------+
only showing top 5 rows



In [12]:
#filter for last 4 weeks
last4weeksfilter = left_join.filter(left_join.date > left_join.last4weeks)
last4weeksfilter.show(5)

+---+----------+---------+-------------------+----------+----------+----------+
| id|      date|    class|          max(date)|last4weeks|last6weeks|last8weeks|
+---+----------+---------+-------------------+----------+----------+----------+
|  1|2019-01-05|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  1|2019-01-07| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  3|2019-01-06|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  2|2019-01-06|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
|  1|2019-01-06|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
+---+----------+---------+-------------------+----------+----------+----------+
only showing top 5 rows



In [13]:
last4weeks_activity = last4weeksfilter.groupBy("id").pivot("class").count()
last4weeks_activity.na.fill(value=0).show(5)

+---+-----+--------+---------+------+-------------+
| id|Bronx|Brooklyn|Manhattan|Queens|Staten Island|
+---+-----+--------+---------+------+-------------+
|  7|    9|     303|      335|    83|            2|
| 15|    0|      42|       24|    15|            0|
|200|    0|       2|        1|     0|            0|
| 11|    0|       7|        5|     1|            0|
| 29|    0|      17|       52|    12|            0|
+---+-----+--------+---------+------+-------------+
only showing top 5 rows



In [14]:
last4weeksfilter.filter(last4weeksfilter.id == 11).show()

+---+----------+---------+-------------------+----------+----------+----------+
| id|      date|    class|          max(date)|last4weeks|last6weeks|last8weeks|
+---+----------+---------+-------------------+----------+----------+----------+
| 11|2019-01-06|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-05| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-06|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-05| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-04| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-04|   Queens|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-03| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-07|Manhattan|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-01| Brooklyn|2019-01-07 00:00:00|2018-12-08|2018-11-23|2018-11-08|
| 11|2019-01-01|Manhattan|2019-01-07 00:

In [15]:
last4weeksfilter.filter(last4weeksfilter.id == 11).groupby('class').count().show()

+---------+-----+
|    class|count|
+---------+-----+
|   Queens|    1|
| Brooklyn|    7|
|Manhattan|    5|
+---------+-----+

